In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.insert(0, 'lib/reid')
import torchreid

import numpy as np
np.set_printoptions(suppress=True)     
import os
import cv2
import random
import pickle
import joblib
import time
import sklearn.metrics
import gurobipy as gp
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.set_printoptions(sci_mode=False)
from torch_geometric.data import Data
from lib.tracking import Tracker
from lib.qpthlocal.qp import QPFunction, QPSolvers
from lib.utils import getIoU, interpolateTrack, interpolateTracks
#from lib.qpthlocal.qp_cuda import QPFunction, QPSolvers, make_gurobi_model #If we want to fine-tune reid

import matplotlib.pyplot as plt
plt.style.use('bmh')
%matplotlib inline

torch.random.manual_seed(123)
np.random.seed(123)

# Prepare data used for training

In [2]:
train_data_list_full = []
val_data_list_full = []
for file in os.listdir('data/train_data/'):
    file_name = 'data/train_data/' + file
    with open(file_name, 'rb') as f:
        data_list = pickle.load(f)
        
    if file.startswith('MOT16-09') or file.startswith('MOT16-13'):
        val_data_list_full = val_data_list_full + data_list
    else:
        train_data_list_full = train_data_list_full + data_list
        
print('Total {} samples in training set, {} in validation set'.format(
                     len(train_data_list_full),len(val_data_list_full)))

train_data_list = []
val_data_list = []
for ind in range(len(train_data_list_full)):
    if train_data_list_full[ind].x.shape[0] < 200: #Avoid too big graph
        train_data_list.append(train_data_list_full[ind])
    else:
        continue
        
for ind in range(len(val_data_list_full)):
    if val_data_list_full[ind].x.shape[0] < 200:
        val_data_list.append(val_data_list_full[ind])
    else:
        continue
        
print('Used {} samples in training set, {} in validation set'.format(len(train_data_list), len(val_data_list)))

Total 748 samples in training set, 319 in validation set
Used 460 samples in training set, 208 in validation set


# Train

In [3]:
class Net(nn.Module): 
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Sequential(nn.Linear(6,6), nn.ReLU(), nn.Linear(6,1))
    def forward(self, data):
        x = self.fc(data.edge_attr)
        x = nn.Sigmoid()(x)
        return x
    
net = Net()
tracker = Tracker(net)
train_list, val_list = [], []
gamma = 0.1

optimizer = torch.optim.Adam(tracker.parameters(), lr=1e-3, weight_decay=1e-4, betas=(0.9, 0.999))
#optimizer = torch.optim.Adam(model.parameters(), lr=3e-3, weight_decay=1e-4, betas=(0.9, 0.999))

for epoch in range(1, 25):
    np.random.shuffle(train_data_list) #Shuffle data list
    for iteration in range(len(train_data_list)):
        
        train_data = train_data_list[iteration]
        A_eq, b_eq, A_ub, b_ub, x_gt, tran_indicator = tracker.build_constraint(train_data, max_frame_gap=1)
        A, b, G, h = tracker.build_constraint_torch(A_eq, b_eq, A_ub, b_ub)

        num_nodes = int(A_eq.shape[0] / 2)
        Q = gamma * torch.eye(G.shape[1])

        prob = tracker.net(train_data)
        prob = torch.clamp(prob, min=1e-7, max=1-1e-7)
        prob_numpy = prob.detach().numpy()
        auc = sklearn.metrics.roc_auc_score(x_gt[num_nodes*3: ], prob_numpy)

        c_entry, c_exit, c_det = torch.ones(num_nodes), torch.ones(num_nodes), -1 * torch.ones(num_nodes)
        c_pred = -1 * torch.log(prob).squeeze()
        c_pred = torch.cat([c_det, c_entry, c_exit, c_pred])

        model_params_quad = tracker.make_gurobi_model_tracking(G.numpy(), h.numpy(), A.numpy(), 
                                                               b.numpy(), Q.numpy())
        x = QPFunction(verbose=False, solver=QPSolvers.GUROBI, maxIter=50, 
                       model_params=model_params_quad)(Q,c_pred,G,h,A,b)

        loss = nn.MSELoss()(x, torch.from_numpy(x_gt).float().t())
        loss_edge = nn.MSELoss()(x[:, num_nodes*3:], torch.from_numpy(x_gt[num_nodes*3:]).float().t())

        const_cost = 1
        c_ = 0 * x_gt[num_nodes*3:] + const_cost * (1 - x_gt[num_nodes*3:])
        c_ = c_.squeeze()
        c_gt = torch.cat([c_pred[:num_nodes*3], torch.from_numpy(c_).float()]) #this is the ground truth cost

        obj_gt = c_gt @ torch.from_numpy(x_gt.squeeze()).float()
        obj_pred = c_pred @ x.squeeze()

        bce = nn.BCELoss()(prob, torch.from_numpy(x_gt[num_nodes*3:]).float())
        x_sol = tracker.linprog(c_pred.detach().numpy(), A_eq, b_eq, A_ub, b_ub)
        ham_loss = sklearn.metrics.hamming_loss(x_gt, x_sol)
        train_list.append((loss.item(), loss_edge.item(), auc, bce.item()))
        
        print('Train Epoch {} iter {}/{}, Objective {:.2f}/{:.2f}, mse {:.4f} mse edge {:.4f} ce {:.3f} \
auc {:.3f} hamming {:.3f}'.format(epoch, iteration, len(train_data_list), obj_pred.item(), 
                                  obj_gt.item(),loss.item(), loss_edge.item(), bce.item(), auc, ham_loss))
        optimizer.zero_grad()
        loss_edge.backward()
        #bce.backward()
        optimizer.step()
    torch.save(tracker.net.state_dict(), 'ckpt/epoch_{}.pth'.format(epoch))
    
    np.random.shuffle(val_data_list)
    for iteration in range(len(val_data_list)):
        
        val_data = val_data_list[iteration]
        A_eq, b_eq, A_ub, b_ub, x_gt, tran_indicator = tracker.build_constraint(val_data, 1)
        A, b, G, h = tracker.build_constraint_torch(A_eq, b_eq, A_ub, b_ub)
        Q = gamma*torch.eye(G.shape[1])
        num_nodes = int(A_eq.shape[0] / 2)
        
        with torch.no_grad():
            prob = tracker.net(val_data)
            prob = torch.clamp(prob, min=1e-7, max=1-1e-7)
        
        prob_numpy = prob.detach().numpy()
        auc = sklearn.metrics.roc_auc_score(x_gt[num_nodes*3: ], prob_numpy)

        c_entry, c_exit, c_det = torch.ones(num_nodes), torch.ones(num_nodes), -1 * torch.ones(num_nodes)
        c_pred = -1 * torch.log(prob).squeeze()
        c_pred = torch.cat([c_det, c_entry, c_exit, c_pred])

        model_params_quad = tracker.make_gurobi_model_tracking(G.numpy(), h.numpy(), A.numpy(),
                                                               b.numpy(), Q.numpy())
        x = QPFunction(verbose=False, solver=QPSolvers.GUROBI, maxIter=50,
                       model_params=model_params_quad)(Q, c_pred, G, h, A, b)

        loss = nn.MSELoss()(x, torch.from_numpy(x_gt).float().t())
        loss_edge = nn.MSELoss()(x[:, num_nodes*3:], torch.from_numpy(x_gt[num_nodes*3:]).float().t())
        
        const_cost = 1
        c_ = 0 * x_gt[num_nodes*3:] + const_cost * (1 - x_gt[num_nodes*3:])
        c_ = c_.squeeze()
        c_gt = torch.cat([c_pred[:num_nodes*3], torch.from_numpy(c_).float()]) #this is the ground truth cost
        
        obj_gt = c_gt @ torch.from_numpy(x_gt.squeeze()).float()
        obj_pred = c_pred @ x.squeeze()

        bce = nn.BCELoss()(prob, torch.from_numpy(x_gt[num_nodes*3:]).float())
        x_sol = tracker.linprog(c_pred.detach().numpy(), A_eq, b_eq, A_ub, b_ub)
        ham_loss = sklearn.metrics.hamming_loss(x_gt, x_sol)
        val_list.append((loss.item(), loss_edge.item(), auc, bce.item()))
        print('Val Epoch {}, iter {}/{}, Objective {:.2f}/{:.2f}, mse {:.3f} mse edge {:.3f} ce {:.3f}\
auc {:.3f} hamming {:.3f}'.format(epoch, iteration, len(val_data_list), obj_pred.item(), 
                                  obj_gt.item(), loss.item(), loss_edge.item(), bce.item(), auc, ham_loss))

Using license file /home/lishuai/Experiment/gurobi910/gurobi.lic
Academic license - for non-commercial use only - expires 2021-03-09
Train Epoch 1 iter 0/460, Objective -5.39/-80.00, mse 0.1979 mse edge 0.1484 ce 0.853 auc 0.427 hamming 0.201
Train Epoch 1 iter 1/460, Objective -5.77/-73.00, mse 0.1987 mse edge 0.1413 ce 0.845 auc 0.344 hamming 0.199
Train Epoch 1 iter 2/460, Objective -5.79/-62.00, mse 0.2058 mse edge 0.1591 ce 0.835 auc 0.378 hamming 0.206
Train Epoch 1 iter 3/460, Objective -5.23/-157.00, mse 0.1366 mse edge 0.0792 ce 0.955 auc 0.177 hamming 0.138
Train Epoch 1 iter 4/460, Objective -5.26/-161.00, mse 0.1353 mse edge 0.0796 ce 0.965 auc 0.141 hamming 0.136
Train Epoch 1 iter 5/460, Objective -5.79/-111.00, mse 0.1712 mse edge 0.1108 ce 0.882 auc 0.244 hamming 0.172
Train Epoch 1 iter 6/460, Objective -5.76/-72.00, mse 0.2106 mse edge 0.1545 ce 0.861 auc 0.277 hamming 0.213
Train Epoch 1 iter 7/460, Objective -5.80/-118.00, mse 0.1595 mse edge 0.0982 ce 0.881 auc 0.3

KeyboardInterrupt: 